In [19]:
import pandas as pd

wine = pd.read_csv("https://bit.ly/wine_csv_data")

input_data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']


##### Validation Set
- 모델 학습 이후의 성능 평가는 검증셋을 통해 한다.

In [20]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    input_data, target, test_size=0.2, random_state=42
)

print(train_input.shape)
print(test_input.shape)

# 훈련셋에서 검증셋을 다시 분리
train_input, val_input, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)
print('--Make Validation Set--')
print(train_input.shape)
print(val_input.shape)

(5197, 3)
(1300, 3)
--Make Validation Set--
(4157, 3)
(1040, 3)


In [21]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.8634615384615385


👉 훈련셋에 오버피팅되어있을 확률이 높음

- 매개변수를 바꿔서 좀 더 좋은 모델을 찾아야 한다.

---
# 교차 검증(Cross Validation)
- 뭐가 됐든 일단 훈련 데이터를 많이 사용해야 좋은 모델이 만들어짐.
    - 검증셋을 만드느라 훈련 데이터가 줄어들었음
    - 그렇다고 검증셋의 양을 줄이면 측정하는 모델의 성능이 들쑥날쑥해질것임
    - 이를 해결하기 위한 방법이 검증셋의 양은 그대로 두고, 서로 다른 여러 검증셋을 통해 모델 성능을 측정하는 방식(Cross Validation)
        - K-Fold Cross Validation: 훈련 셋을 K개로 쪼개어, K개의 검증셋을 순차적으로 각각의 검증셋으로 활용하는 방식

In [22]:
from sklearn.model_selection import cross_validate

train_input, test_input, train_target, test_target = train_test_split(
    input_data, target, test_size=0.2, random_state=42
)

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00499582, 0.00505209, 0.0048418 , 0.005481  , 0.00513792]), 'score_time': array([0.00068092, 0.00076199, 0.00059009, 0.00076389, 0.00070906]), 'test_score': array([0.86826923, 0.85      , 0.87391723, 0.85755534, 0.8373436 ])}


- 각 value값이 5개씩 있는것으로 볼 때, 기본 폴드값은 5라는것을 알 수 있음
- `fit_time`은 훈련시간, `score_time`은 검증하는 시간을 의미함
- `test_score`는 각각의 테스트셋에 대한 점수로, 이 값의 평균값이 모델의 최종 성능임

In [23]:
import numpy as np

print(np.mean(scores['test_score']))

0.8574170800325757


- 주의해야하는 점은, `cross_validate()`는 훈련셋을 섞어서 폴드를 나누는게 아니라, 그냥 있는대로 나눈다.
- 앞선 `train_test_split()`은 처음에 데이터를 섞고 나서 데이터를 분리했기 때문에 상관 없는데, 교차 검증에서 데이터를 섞기 위해서는 분할기(splitter)를 지정해야한다.
    - `cross_validate()`함수는 일반적으로 회귀 모델의 경우 `KFold`분할기, 분류 모델의 경우 타겟 클래스를 골고루 나누기 위해 `StratifiedKFold`를 사용한다. 사용은 아래와 같다.

In [24]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8570324646479603


In [25]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8587672298799467


---
# 하이퍼 파라미터 튜닝
- 모델마다 여러 하이퍼 파라미터값이 존재하는데, 각 하이퍼 파라미터값에 대한 모델의 성능은 하이퍼 파라미터들의 복합적인 관계를 통해 좌우되기때문에 복잡함
- 사이킷런에서는 최적의 하이퍼 파라미터 조합을 찾기 위한 클래스로 그리드서치(`GridSearch`)를 제공

In [26]:
from sklearn.model_selection import GridSearchCV

h_params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), h_params, n_jobs=-1)

- `GridSearchCV`의 cv(cross validation) 기본값은 5이므로, 교차 검증은 총 25번 이루어짐. 즉 25개의 모델을 훈련한다고 볼 수 있음
- `n_jobs`는 병렬 실행에 사용할 CPU코어의 수로, 기본값은 1이고 -1로 설정하면 시스템의 모든 코어를 사용함
- 그리드 서치의 파라미터에는 반드시 딕셔너리 형태(`{}`) 또는 딕셔너리들의 리스트(`[{}, {}, ..]`)형태로 들어가야함

In [27]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- 최적의 하이퍼파라미터값을 갖는 모델로 설정하고, 점수를 다시 내보자.
- 최적의 하이퍼파라미터 값이 몇인지 직접 확인할수도 있다. (`best_params_`)

In [28]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


- 각 하이퍼 파라미터 조합 별 점수도 확인할 수 있음 (점수는 validaion set으로 지정된 폴드에 대한 점수. 즉 테스트 점수가 아닌 validation set에 대한 점수)
- 직접 눈으로 보고 고르는것보다, `best_index_`속성을 통해 가장 높은 값을 갖는 인덱스를 뽑아낼 수도 있음

In [29]:
print(gs.cv_results_['mean_test_score'])
print(gs.best_index_)

print(gs.cv_results_['params'][gs.best_index_])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]
0
{'min_impurity_decrease': 0.0001}


---
# 하이퍼 파라미터 여러 개로 학습해보기
- DecisionTree의 `min_impurity_decrease`는 노드를 분할하기 위한 불순도 감소 최소량이다. 여기서 추가로 `max_depth`를 통해 트리의 깊이를 제한하고, `min_samples_split`으로 노드를 나누기 위한 최소 샘플의 수도 정의해보자.
    - 그리드 서치는 아무 모델의 하이퍼 파라미터를 막 가져다 쓸 수 있는게 아니라, estimator(현재는 DecisionTreeClassifier)의 `__init__` 또는 `get_params()`에서 확인 가능한 이름 중 하나여야한다. (내부에서 `추정기.set_params(**params)`를 호출하기 때문)


In [32]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

- `arange()`는 `range()`와 달리 실수값을 만들 수 있어서 좋음
- 하이퍼 파라미터를 저렇게 조합하면, 총 교차 검증 횟수는 $9*15*10 =1350$, 만들어지는 모델의 수는 6750

In [33]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


베스트 하이퍼 파라미터 조합과, 각 폴드에 대한 점수 평균값

In [34]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}
0.8683865773302731


---
# 랜덤 서치
- 하이퍼 파라미터 값이 수치일 때, 범위나 간격 등을 미리 정하기 어려울 수 있음
- 또한 너무 많은 매개변수 조건이 있어서 그리드 서치의 수행 시간이나 연산량이 많아질 수 있음
    - 이럴 경우 랜덤서치를 활용할 수 있음
- 랜덤서치는 하이퍼 파라미터 값의 목록이 아니라, '하이퍼 파라미터를 샘플링할 수 있는 확률 분포 객체'를 전달함

In [35]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([5, 8, 5, 9, 0, 9, 4, 2, 8, 9])

- 복원추출임
- 10개라서 골고루 추출하는지 아닌지 잘 모르겠는데, 추출 횟수를 1000회로 늘려보자. 각 샘플을 어느정도 골고루 추출하는 것을 볼 수 있다.

In [37]:
samples, counts = np.unique(rgen.rvs(1000), return_counts=True)
print(samples)
print(counts)

[0 1 2 3 4 5 6 7 8 9]
[102  99  89 103  89 112 111  97 100  98]


In [38]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.04912197, 0.68435655, 0.95537668, 0.54057815, 0.87302736,
       0.98017733, 0.40827996, 0.41611985, 0.30496068, 0.57900106])

### 랜덤 서치로 아까처럼 하이퍼 파라미터의 조합 찾아보기

In [39]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

print(rs.best_params_) # 최적의 파라미터 조합
print(np.max(rs.cv_results_['mean_test_score'])) # 그 떄의 점수

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [41]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86


importance값은 트리 전체(루트부터 리프 노드까지)에 대해, 해당 피쳐를 기반으로 분할했을 때 불순도 감소가 얼마나 일어났는지를 기반으로 계산한다.